In [1]:
import openai
import json
from apitable import Apitable
import os
import json
import requests

from tenacity import retry, wait_random_exponential, stop_after_attempt

from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [4]:
apitable_api_token = os.getenv("APITABLE_API_TOKEN")
apitable = Apitable(token=apitable_api_token)

In [5]:
def trans_key(field_key_map, key: str):
    """
    When there is a field mapping, convert the mapped key to the actual key
    """
    if key in ["_id", "recordId"]:
        return key
    if field_key_map:
        _key = field_key_map.get(key, key)
        return _key
    return key

def query_parse(field_key_map, **kwargs) -> str:
    query_list = []
    for k, v in kwargs.items():
        # Handling null
        if v is None:
            v = "BLANK()"
        # Handling string
        elif isinstance(v, str):
            v = f'"{v}"'
        elif isinstance(v, bool):
            v = "TRUE()" if v else "FALSE()"
        # Handling array type values, multiple select, members?
        elif isinstance(v, list):
            v = f'"{", ".join(v)}"'
        query_list.append(f"{{{trans_key(field_key_map, k)}}}={v}")
    if len(query_list) == 1:
        return query_list[0]
    else:
        qs = ",".join(query_list)
        return f"AND({qs})"

def get_spaces():
    spaces = apitable.spaces.all()
    return [json.loads(space.json()) for space in spaces]

def get_nodes(space_id: str):
    nodes = apitable.space(space_id=space_id).nodes.all()
    return [json.loads(node.json()) for node in nodes]

def get_fields(datasheet_id: str):
    fields = apitable.datasheet(datasheet_id).fields.all()
    return [json.loads(field.json()) for field in fields]

# def create_fields(space_id: str, datasheet_id: str, field_data: dict):
#     field = (
#         apitable.space(space_id)
#         .datasheet(datasheet_id)
#         .fields.create(field_data)
#     )
#     return field.json()

def get_records(query: dict):
    datasheet_id = query["datasheet_id"]
    dst = apitable.datasheet(datasheet_id)
    query_kwargs = {}
    if "filter_condition" in query.keys():
        query_formula = query_parse(query["filter_condition"])
        query_kwargs["filterByFormula"] = query_formula
    if "sort_condition" in query.keys():
        query_kwargs["sort"] = query["sort_condition"]
    if "maxRecords_condition" in query.keys():
        query_kwargs["maxRecords"] = query["maxRecords_condition"]
    records = dst.records.all(**query_kwargs)
    return [record.json() for record in records]

In [6]:
functions = [
    {
        "name": "get_spaces",
        "description": "This function retrieves all spaces accessible to the user. If the user's prompt does not specify a space_id, this function should be used initially.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_prompt": {
                    "type": "string",
                    "description": "User's question",
                },
            },
        }
    },
    {
        "name": "get_nodes",
        "description": "This function retrieves all datasheets, mirrors, dashboards, folders, and forms.",
        "parameters": {
            "type": "object",
            "properties": {
                "space_id": {
                    "type": "string",
                    "description": "The ID of the spaces to retrieve data from.",
                },
            },
            "required": ["space_id"],
        }
    },
    {
        "name": "get_fields",
        "description": "This function can search for fields in a datasheet",
        "parameters": {
            "type": "object",
            "properties": {
                "datasheet_id": {
                    "type": "string",
                    "description": "The ID of the datasheet to retrieve fields from.",
                },
            },
            "required": ["datasheet_id"],
        }
    },
    {
        "name": "get_records",
        "description": "This function can retrieve data from a datasheet.",
        "parameters": {
            "type": "object",
            "properties": {
                "datasheet_id": {
                    "type": "string",
                    "description": "The ID of the datasheet to retrieve records from.",
                },
                "filter_condition": {
                    "type": "object",
                    "description": """
                        Find records that meet specific conditions.
                        This object should contain a key-value pair where the key is the field name and the value is the lookup value. For instance: {"title": "test"}.
                        """,
                },
                "sort_condition": {
                    "type": "array",
                    "description": "Sort returned records by specific field.",
                    "items":{
                        "type": "object",
                        "properties": {
                            "field": {
                                "type": "string",
                                "description": "Field name",
                            },
                            "order": {
                                "type": "string",
                                "description": "Sort order",
                                "enum": ["desc", "asc"]
                            }
                        }
                    }
                },
                "maxRecords_condition": {
                    "type": "number",
                    "description": """
                        Limit the number of returned values.
                        """,
                },
            },
            "required": ["datasheet_id"],
        },
    }
]

In [7]:
def chat_completion_with_function_execution(messages, functions=None):
    """This function makes a ChatCompletion API call and if a function call is requested, executes the function"""
    try:
        response = chat_completion_request(messages, functions)
        full_message = response.json()["choices"][0]
        if full_message["finish_reason"] == "function_call":
            print(f"Function generation requested, calling function")
            return call_function(messages, full_message)
        else:
            print(f"Function not required, responding to user")
            return response.json()
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return response


def call_function(messages, full_message):
    """Executes function calls using model generated function arguments."""

    query = json.loads(full_message["message"]["function_call"]["arguments"])
    a =full_message["message"]["function_call"]["name"]
    print(f"Function is {a}")
    print(f"Prepped query is {query}")

    # We'll add our one function here - this can be extended with any additional functions
    try:
        if full_message["message"]["function_call"]["name"] == "get_spaces":
            results = get_spaces()
        elif full_message["message"]["function_call"]["name"] == "get_nodes":
            results = get_nodes(query['space_id'])
        elif full_message["message"]["function_call"]["name"] == "get_fields":
            results = get_fields(query['datasheet_id'])
        elif full_message["message"]["function_call"]["name"] == "get_records":
            results = get_records(query)
        else:
            raise Exception("Function does not exist and cannot be called")
    except Exception as e:
        results = f"Function call failed with exception: {e}"

    messages.append(
        {"role": "function", "name": full_message["message"]["function_call"]["name"], "content": str(results)}
    )
    
    try:
        response = chat_completion_request(messages)
        return response.json()
    except Exception as e:
        print(type(e))
        print(e)
        raise Exception("Function chat request failed")


In [8]:
agent_system_message = """You are APItableGPT, a helpful assistant who gets answers to user questions by APITable Functions with APITable data.
Provide as many details as possible to your users, if you need more information to answer questions, please use functions to get the information you need.
Begin!"""

conversation = Conversation()
conversation.add_message("system", agent_system_message)
conversation.add_message(
    "user", "What spaces do I have"
)

In [9]:
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_spaces
Prepped query is {}
You have the following spaces:

1. xukecheng's Space (ID: spctqtTZpssYw): You are the admin of this space.
2. APITable Ltd. (ID: spcS0eZxZ8mSA): Admin status is not available.
3. Gmail Space (ID: spcGSVizcwRYF): Admin status is not available.
4. test03-bug验证升级 (ID: spcV5VkzU71Lf): Admin status is not available.

Please let me know if there's anything else I can help with!


In [10]:
conversation.add_message(
    "user", "Tell me the latest value of APITable MAU in xukecheng's space"
)
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_nodes
Prepped query is {'space_id': 'spctqtTZpssYw'}
To provide you with the latest value of APITable MAU (Monthly Active Users) in xukecheng's space, I need to access the "APITable MAUs" datasheet.

Here are the available nodes in xukecheng's space:
1. Project management (Datasheet)
2. Make Test (Folder)
3. Form (Form)
4. AITEST (Datasheet)
5. email_oss (Datasheet)
6. APITable MAUs (Datasheet)

Since "APITable MAUs" is the datasheet we need, I'll retrieve the latest value for you.


In [11]:
conversation.add_message(
    "user", "Tell me the latest value of APITable MAU in xukecheng's space"
)
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_records
Prepped query is {'datasheet_id': 'dsti6VpNpuKQpHVSnh', 'sort_condition': {'field': 'Date', 'order': 'desc'}, 'maxRecords_condition': 1}
Apologies for the inconvenience. Let me fix the issue and retrieve the latest value of APITable MAU in xukecheng's space.


In [12]:
conversation.add_message(
    "user", "Tell me the latest value of APITable MAU in xukecheng's space"
)
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_records
Prepped query is {'datasheet_id': 'dsti6VpNpuKQpHVSnh', 'sort_condition': [{'field': 'date', 'order': 'desc'}], 'maxRecords_condition': 1}
I apologize for the inconvenience. It seems that there is an issue retrieving the latest value of APITable MAU in xukecheng's space. To fix this, I need to sort the data based on the "date" field. Let me try again.


In [13]:
conversation.add_message(
    "user", "Tell me the latest value of APITable MAU in xukecheng's space"
)
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_fields
Prepped query is {'datasheet_id': 'dsti6VpNpuKQpHVSnh'}
The APITable MAU datasheet in xukecheng's space has the following fields:

1. Title (SingleText): Represents the title of the record.
2. MAU (Number): Represents the Monthly Active Users value.
3. Created time (CreatedTime): Represents the time when the record was created.
4. 是否为整百 (Formula): Represents whether the MAU value is a multiple of 100.

To retrieve the latest value of APITable MAU, I will sort the records based on the "Created time" field and retrieve the record with the most recent "Created time".


In [14]:
conversation.add_message(
    "user", "Tell me the latest value of APITable MAU in xukecheng's space"
)
chat_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
conversation.add_message("assistant", assistant_message)
print(assistant_message)

Function generation requested, calling function
Function is get_records
Prepped query is {'datasheet_id': 'dsti6VpNpuKQpHVSnh', 'sort_condition': [{'field': 'Created time', 'order': 'desc'}], 'maxRecords_condition': 1}
The latest value of APITable MAU in xukecheng's space is 331. This value was recorded at a timestamp of 1686153676721.

If you have any more questions, feel free to ask!
